In [35]:
import pandas as pd
import tensorflow as tf
import sklearn
import scikeras

In [36]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from tensorflow.keras import backend as k
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from scikeras.wrappers import KerasRegressor


In [37]:
base = pd.read_csv('autos.csv', encoding='ISO-8859-1')


In [38]:
base = base.drop('dateCrawled', axis=1)
base = base.drop('dateCreated', axis=1)
base = base.drop('nrOfPictures', axis=1)
base = base.drop('postalCode', axis=1)
base = base.drop('lastSeen', axis=1)
base = base.drop('name', axis=1)
base = base.drop('seller', axis=1)
base = base.drop('offerType', axis=1)


In [39]:
base = base[base['price'] > 10]
base = base.loc[base['price'] < 350000]

In [40]:
valores = {'vehicleType': 'limousine',
           'gearbox': 'manuell',
           'model': 'golf',
           'fuelType': 'benzin',
           'notRepairedDamage': 'nein'}
base = base.fillna(value = valores)

In [41]:
X = base.iloc[:, 1:12].values
y = base.iloc[:, 0].values

In [42]:
onehotencoder = ColumnTransformer(
    transformers=[("OneHot", OneHotEncoder(), [0, 1, 3, 5, 8, 9, 10])],
      remainder='passthrough')
X = onehotencoder.fit_transform(X).toarray()

In [43]:
X.shape

(359291, 316)

In [44]:
def create_network():
    k.clear_session()
    regressor = Sequential([
        tf.keras.layers.InputLayer(shape=(316,)),
        tf.keras.layers.Dense(units=158, activation='relu'),
        tf.keras.layers.Dense(units=158, activation='relu'),
        tf.keras.layers.Dense(units=1, activation='linear')])
    regressor.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
    return regressor

In [45]:
regressor = KerasRegressor(model=create_network, epochs=100, batch_size=300)

In [46]:
results = cross_val_score(estimator=regressor, X=X, y=y, cv=5, scoring='neg_mean_absolute_error')


Epoch 1/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 4409.7983 - mean_absolute_error: 4409.7983
Epoch 2/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 3520.7542 - mean_absolute_error: 3520.7542
Epoch 3/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 3341.3723 - mean_absolute_error: 3341.3723
Epoch 4/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 3106.9478 - mean_absolute_error: 3106.9478
Epoch 5/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 2954.3508 - mean_absolute_error: 2954.3508
Epoch 6/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 2835.8630 - mean_absolute_error: 2835.8633
Epoch 7/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 2765.7737 - mean_absolute_error: 2765.7737
Epoch 8/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 2687.6086 - mean_absolute_error: 2687.6086
Epoch 9/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 2684.4033 - mean_absolute_error: 2684.4033
Epoch 10/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 2

In [48]:
abs(results)

array([2305.52540423, 2268.02108481, 2239.10477843, 2288.89480632,
       2162.68883187])

In [49]:
abs(results.mean())

2252.8469811336645

In [50]:
abs(results.std())

50.24351778348842